In [1]:
# Load some libraries

import pyodbc
from dotenv import dotenv_values
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.impute import SimpleImputer
import re
import warnings

warnings.filterwarnings('ignore')

In [2]:
# # Load environment variables from .env file into a dictionary

# env_var = dotenv_values('.env')

# # Get the values for the credentials you set in the '.env' file

# database = env_var.get('DATABASE')
# username = env_var.get('USERNAME')
# password = env_var.get('PASSWORD')
# server = env_var.get('SERVER')


# connection_string = f"DRIVER={{SQL Server}}; SERVER={server};DATABASE={database};UID={username};PWD={password}"

In [3]:
# Get the values for the database credentials set in the '.env' file
env_var = {key.upper(): value for key, value in dotenv_values('.env').items()}

# Unpack the values from the dictionary
database, username, password, server = (env_var.get(key) for key in ['DATABASE', 'USERNAME', 'PASSWORD', 'SERVER'])

connection_string = f"DRIVER={{SQL Server}}; SERVER={server};DATABASE={database};UID={username};PWD={password}"

In [4]:
# Connect to database

connection = pyodbc.connect(connection_string)

In [5]:
# Loading the data for 2020

query2020 = "select * from dbo.LP1_startup_funding2020"

data2020 = pd.read_sql(query2020, connection)

In [6]:
# Loading the data for 2021

query2021 = "select * from dbo.LP1_startup_funding2021"

data2021 = pd.read_sql(query2021, connection)

In [7]:
# Loading data for 2021 & 2019


data2018 = pd.read_csv('startup_funding2018.csv')
data2019 = pd.read_csv('startup_funding2019.csv')

## Data Inspection

### 2021 Data

In [8]:
# Inspecting 2021 data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
data2021

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
0,Unbox Robotics,2019.0,Bangalore,AI startup,Unbox Robotics builds on-demand AI-driven ware...,"Pramod Ghadge, Shahid Memon","BEENEXT, Entrepreneur First","$1,200,000",Pre-series A
1,upGrad,2015.0,Mumbai,EdTech,UpGrad is an online higher education platform.,"Mayank Kumar, Phalgun Kompalli, Ravijot Chugh,...","Unilazer Ventures, IIFL Asset Management","$120,000,000",None
2,Lead School,2012.0,Mumbai,EdTech,LEAD School offers technology based school tra...,"Smita Deorah, Sumeet Mehta","GSV Ventures, Westbridge Capital","$30,000,000",Series D
3,Bizongo,2015.0,Mumbai,B2B E-commerce,Bizongo is a business-to-business online marke...,"Aniket Deb, Ankit Tomar, Sachin Agrawal","CDC Group, IDG Capital","$51,000,000",Series C
4,FypMoney,2021.0,Gurugram,FinTech,"FypMoney is Digital NEO Bank for Teenagers, em...",Kapil Banwari,"Liberatha Kallat, Mukesh Yadav, Dinesh Nagpal","$2,000,000",Seed
5,Urban Company,2014.0,New Delhi,Home services,Urban Company (Formerly UrbanClap) is a home a...,"Abhiraj Singh Bhal, Raghav Chandra, Varun Khaitan",Vy Capital,"$188,000,000",None
6,Comofi Medtech,2018.0,Bangalore,HealthTech,Comofi Medtech is a healthcare robotics startup.,Gururaj KB,"CIIE.CO, KIIT-TBI","$200,000",None
7,Qube Health,2016.0,Mumbai,HealthTech,India's Most Respected Workplace Healthcare Ma...,Gagan Kapur,Inflection Point Ventures,Undisclosed,Pre-series A
8,Vitra.ai,2020.0,Bangalore,Tech Startup,Vitra.ai is an AI-based video translation plat...,Akash Nidhi PS,Inflexor Ventures,Undisclosed,None
9,Taikee,2010.0,Mumbai,E-commerce,"Taikee is the ISO-certified, B2B e-commerce pl...","Nidhi Ramachandran, Sachin Chhabra",None,"$1,000,000",None


## Issues arising from data2021:

* Founded column is a float. It has to be a date

* Some Amounts have $undisclosed, Undisclosed and undisclosed. We may treat them as missing values

* The Amounts and Stage for FanPlay company are interchanged

* At index 242, 256, 257, and 545, the amount appears at the investor’s column and then stage at the amount column

* Little Leap campany at index 538 has Ah! Ventures(investors) instead of amount and then amount at stage column. Also, ‘Holistic Development Programs for children in …; should be   replaced with Vishal Gupta as founder.

* BHyve company has part of ‘what_it_does’ at founders cl then founders at investor cl and investors at amount col ……. It also had investor’s name as ITO Angel Network instead of JITO Angel Network

* Some amounts are separated by ‘,’ some too have ‘$$’ preceding them and some only ‘$’ as amount…this is for EventBeep, MPL

* Amount for Godamwale is misspelt as 1000000\t#REF! instead of 1000000 and is at investor col with stage rather taking its place. Also, the investors are **Capt. Anand Aryamane**

* for index 1100-Sochcast company, Heaquarter is replaced with ‘Online Media\t#REF!’ and sector
* There are some duplicates

### Resolution

Delete duplicates

In [9]:
# Confirm duplicated records

data2021 = data2021.drop_duplicates()
data2021.shape

(1190, 9)

In [10]:
# Check data shape before deletion
data2021.shape

(1190, 9)

Deleting rows with anomalous values


In [11]:
# Function to drop anomalous rows


def drop_rows_by_column_values(dataframe, column_name, value):
    """
    Drops rows from the specified DataFrame based on a specific column value.

    Parameters:
        dataframe (pandas.DataFrame): The DataFrame to modify.
        column_name (str): The name of the column to check for the specified value.
        value: The value to match and drop rows based on the specified column.

    Returns:
        None

    Description:
        This function removes rows from the specified DataFrame where the specified column
        matches the provided value. The rows are dropped in-place, modifying the DataFrame directly.

        Parameters:
        - dataframe (pandas.DataFrame): The DataFrame to modify.
        - column_name (str): The name of the column to check for the specified value.
        - value: The value to match and drop rows based on the specified column.

        Returns:
        - None

        Example usage:
        drop_rows_by_column_value(data2021, 'Company_Brand', 'BrandXYZ')

        This example will drop all rows from the 'data2021' DataFrame where the 'Company_Brand'
        column contains the value 'BrandXYZ'.
    """
    dataframe.drop(dataframe[dataframe[column_name].isin(value)].index, inplace=True)

In [12]:
# Invoke function to drop the list of rows

drop_rows_by_column_values(data2021, 'Company_Brand', ['Little Leap', 'Godamwale', 'Sochcast', 'BHyve', 'Fullife Healthcare', 'MoEVing', 'AdmitKard', 'MYRE Capital'])


In [13]:
# Making sure the rows were deleted

data2021.shape

(1182, 9)

In [14]:
# Re-arrange indices for the data frame

data2021.reset_index(drop=True, inplace=True)

In [15]:
# Check for re-indexing

data2021.tail()

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
1177,Gigforce,2019.0,Gurugram,Staffing & Recruiting,A gig/on-demand staffing company.,"Chirag Mittal, Anirudh Syal",Endiya Partners,$3000000,Pre-series A
1178,Vahdam,2015.0,New Delhi,Food & Beverages,VAHDAM is among the world’s first vertically i...,Bala Sarda,IIFL AMC,$20000000,Series D
1179,Leap Finance,2019.0,Bangalore,Financial Services,International education loans for high potenti...,"Arnav Kumar, Vaibhav Singh",Owl Ventures,$55000000,Series C
1180,CollegeDekho,2015.0,Gurugram,EdTech,"Collegedekho.com is Student’s Partner, Friend ...",Ruchir Arora,"Winter Capital, ETS, Man Capital",$26000000,Series B
1181,WeRize,2019.0,Bangalore,Financial Services,India’s first socially distributed full stack ...,"Vishal Chopra, Himanshu Gupta","3one4 Capital, Kalaari Capital",$8000000,Series A


### Dealing with the 'Founded' Col

In [16]:
# Imputing missing values

array = data2021['Founded'].values.reshape(-1,1)
imputer = SimpleImputer(strategy='most_frequent')

imputer.fit(array)

data2021['Founded'] = imputer.transform(array)

In [17]:
data2021['Founded'].unique()

array([2019., 2015., 2012., 2021., 2014., 2018., 2016., 2020., 2010.,
       2017., 1993., 2008., 2013., 1999., 1989., 2011., 2009., 2002.,
       1994., 2006., 2000., 2007., 1978., 2003., 1998., 1991., 1984.,
       2004., 2005., 1963.])

Converting 'Founded' from float to int64

In [18]:

data2021['Founded'] = data2021['Founded'].apply(str).apply(lambda x: x.split('.')[0])
data2021['Founded'] = pd.to_numeric(data2021['Founded'])
data2021.head()

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
0,Unbox Robotics,2019,Bangalore,AI startup,Unbox Robotics builds on-demand AI-driven ware...,"Pramod Ghadge, Shahid Memon","BEENEXT, Entrepreneur First","$1,200,000",Pre-series A
1,upGrad,2015,Mumbai,EdTech,UpGrad is an online higher education platform.,"Mayank Kumar, Phalgun Kompalli, Ravijot Chugh,...","Unilazer Ventures, IIFL Asset Management","$120,000,000",None
2,Lead School,2012,Mumbai,EdTech,LEAD School offers technology based school tra...,"Smita Deorah, Sumeet Mehta","GSV Ventures, Westbridge Capital","$30,000,000",Series D
3,Bizongo,2015,Mumbai,B2B E-commerce,Bizongo is a business-to-business online marke...,"Aniket Deb, Ankit Tomar, Sachin Agrawal","CDC Group, IDG Capital","$51,000,000",Series C
4,FypMoney,2021,Gurugram,FinTech,"FypMoney is Digital NEO Bank for Teenagers, em...",Kapil Banwari,"Liberatha Kallat, Mukesh Yadav, Dinesh Nagpal","$2,000,000",Seed


In [19]:
# Making sure the conversion was done

data2021['Founded'].dtype

dtype('int64')

In [20]:
data2021['Founded'].unique()

array([2019, 2015, 2012, 2021, 2014, 2018, 2016, 2020, 2010, 2017, 1993,
       2008, 2013, 1999, 1989, 2011, 2009, 2002, 1994, 2006, 2000, 2007,
       1978, 2003, 1998, 1991, 1984, 2004, 2005, 1963], dtype=int64)

### Cleaning the 'Amount' col

In [21]:
# Drop additional rows with anomalous values at the Amount col

drop_rows_by_column_values(data2021, 'Amount', ['Upsparks', 'JITO Angel Network, LetsVenture'])

Replacing undisclosed/$undisclosed/$Undisclosed values under the 'Amounts' Column

In [22]:
# Function to replace anomalous amount values with np.nan

def replace_values_with_nan(df, column_name, values_to_replace):
    """
    Replaces specified values in a column of a DataFrame with np.nan.

    Args:
        df (pandas.DataFrame): The DataFrame containing the column to modify.
        column_name (str): The name of the column to replace values in.
        values_to_replace (list or scalar): The value(s) to replace with np.nan. Can be a single value or a list of values.

    Returns:
        pandas.DataFrame: A modified DataFrame with the specified values replaced by np.nan.

    Example:
        # Create a sample DataFrame
        data = {
            'Column1': [1, 2, 3, 4, 5],
            'Column2': ['A', 'B', 'C', 'D', 'E'],
            'Column3': ['X', 'Y', 'Z', 'X', 'Z']
        }

        df = pd.DataFrame(data)

        # Define the column name and values to replace with np.nan
        column_name = 'Column3'
        values_to_replace = ['X', 'Z']

        # Call the replace_values_with_nan function
        df_modified = replace_values_with_nan(df, column_name, values_to_replace)

        # Print the modified DataFrame
        print(df_modified)
    """
    df[column_name] = df[column_name].replace(values_to_replace, np.nan)
    return df


In [23]:
# Invoking the function to replace the missing values with np.nan

data2021 = replace_values_with_nan(data2021, 'Amount', ['$Undisclosed', '$undisclosed', 'undisclosed', 'Undisclosed', 'None'])

# Replacing the undesired characters

data2021['Amount'] = data2021['Amount'].str.replace('[$,]', '')

In [24]:
# Converting the 'Amount' col to float

data2021['Amount'] = pd.to_numeric(data2021['Amount'])

In [25]:
# Checking....

data2021['Amount'].dtype

dtype('float64')

In [26]:
pd.set_option('display.float_format', '{:.1f}'.format)


In [27]:
data2021.describe()

,Founded,Amount
count,1180.0,1040.0
mean,2016.6,172664159.6
std,4.5,4651133125.4
min,1963.0,10000.0
25%,2015.0,1000000.0
50%,2018.0,3700000.0
75%,2020.0,15000000.0
max,2021.0,150000000000.0


Filling missing values in the 'Amount' Col

In [28]:
# Imputing missing values

array = data2021['Amount'].values.reshape(-1,1)
imputer = SimpleImputer(strategy='median')

imputer.fit(array)

data2021['Amount'] = imputer.transform(array)

In [29]:
data2021[data2021['Amount'] == 150000000000.0]

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
73,Alteria Capital,2018,Mumbai,FinTech,Alteria Capital is a Venture debt firm .,Vinod Murali,None,150000000000.0,Debt


In [30]:
top_10_fundings = data2021.groupby('Company_Brand')['Amount'].sum().nlargest(10)

top_10_fundings

Company_Brand
Alteria Capital    150000000000.0
VerSe Innovation     1450000000.0
BYJU'S               1260000000.0
Dream Sports         1240000000.0
Meesho                870000000.0
Zetwerk               870000000.0
OYO                   865000000.0
Swiggy                800000000.0
BharatPe              530200000.0
Ola                   500000000.0
Name: Amount, dtype: float64

In [31]:
data2021.head()

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
0,Unbox Robotics,2019,Bangalore,AI startup,Unbox Robotics builds on-demand AI-driven ware...,"Pramod Ghadge, Shahid Memon","BEENEXT, Entrepreneur First",1200000.0,Pre-series A
1,upGrad,2015,Mumbai,EdTech,UpGrad is an online higher education platform.,"Mayank Kumar, Phalgun Kompalli, Ravijot Chugh,...","Unilazer Ventures, IIFL Asset Management",120000000.0,None
2,Lead School,2012,Mumbai,EdTech,LEAD School offers technology based school tra...,"Smita Deorah, Sumeet Mehta","GSV Ventures, Westbridge Capital",30000000.0,Series D
3,Bizongo,2015,Mumbai,B2B E-commerce,Bizongo is a business-to-business online marke...,"Aniket Deb, Ankit Tomar, Sachin Agrawal","CDC Group, IDG Capital",51000000.0,Series C
4,FypMoney,2021,Gurugram,FinTech,"FypMoney is Digital NEO Bank for Teenagers, em...",Kapil Banwari,"Liberatha Kallat, Mukesh Yadav, Dinesh Nagpal",2000000.0,Seed


In [32]:
data2021['Year_Funded'] = 2021

In [33]:
data2021.head()

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage,Year_Funded
0,Unbox Robotics,2019,Bangalore,AI startup,Unbox Robotics builds on-demand AI-driven ware...,"Pramod Ghadge, Shahid Memon","BEENEXT, Entrepreneur First",1200000.0,Pre-series A,2021
1,upGrad,2015,Mumbai,EdTech,UpGrad is an online higher education platform.,"Mayank Kumar, Phalgun Kompalli, Ravijot Chugh,...","Unilazer Ventures, IIFL Asset Management",120000000.0,None,2021
2,Lead School,2012,Mumbai,EdTech,LEAD School offers technology based school tra...,"Smita Deorah, Sumeet Mehta","GSV Ventures, Westbridge Capital",30000000.0,Series D,2021
3,Bizongo,2015,Mumbai,B2B E-commerce,Bizongo is a business-to-business online marke...,"Aniket Deb, Ankit Tomar, Sachin Agrawal","CDC Group, IDG Capital",51000000.0,Series C,2021
4,FypMoney,2021,Gurugram,FinTech,"FypMoney is Digital NEO Bank for Teenagers, em...",Kapil Banwari,"Liberatha Kallat, Mukesh Yadav, Dinesh Nagpal",2000000.0,Seed,2021


### Cleaning 'Stage' Col

Inspecting the unique values in the 'Stage' Column

In [34]:
data2021['Stage'].unique()

array(['Pre-series A', None, 'Series D', 'Series C', 'Seed', 'Series B',
       'Series E', 'Pre-seed', 'Series A', 'Pre-series B', 'Debt',
       'Bridge', 'Seed+', 'Series F2', 'Series A+', 'Series G',
       'Series F', 'Series H', 'Series B3', 'PE', 'Series F1',
       'Pre-series A1', 'Early seed', 'Series D1', 'Seies A',
       'Pre-series', 'Series A2', 'Series I'], dtype=object)

Check how many records have the null value at the 'Stage' column

In [35]:
data2021['Stage'].isnull().sum()

416

Oops! Pretty much! Let's take a deeper look at those records.

In [36]:
data2021[data2021['Stage'].isnull()]

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage,Year_Funded
1,upGrad,2015,Mumbai,EdTech,UpGrad is an online higher education platform.,"Mayank Kumar, Phalgun Kompalli, Ravijot Chugh,...","Unilazer Ventures, IIFL Asset Management",120000000.0,None,2021
5,Urban Company,2014,New Delhi,Home services,Urban Company (Formerly UrbanClap) is a home a...,"Abhiraj Singh Bhal, Raghav Chandra, Varun Khaitan",Vy Capital,188000000.0,None,2021
6,Comofi Medtech,2018,Bangalore,HealthTech,Comofi Medtech is a healthcare robotics startup.,Gururaj KB,"CIIE.CO, KIIT-TBI",200000.0,None,2021
8,Vitra.ai,2020,Bangalore,Tech Startup,Vitra.ai is an AI-based video translation plat...,Akash Nidhi PS,Inflexor Ventures,3700000.0,None,2021
9,Taikee,2010,Mumbai,E-commerce,"Taikee is the ISO-certified, B2B e-commerce pl...","Nidhi Ramachandran, Sachin Chhabra",None,1000000.0,None,2021
11,FreeStand,2017,New Delhi,B2B service,FreeStand enables FMCG brands to execute track...,"Konark Sharma, Sneh Soni","SucSEED Indovation, IIM Calcutta Innovation Park",100000.0,None,2021
13,Freyr Energy,2014,Hyderabad,Renewable Energy,Freyr Energy is a company that provides full s...,"Radhika Choudary, Saurabh Marda","Impact Partners, C4D Partners",2000000.0,None,2021
14,DealShare,2018,Jaipur,E-commerce,DealShare is a Social Commerce Startup,"Sankar Bora, Sourjyendu Medda, Vineet Rao","Tiger Global Management, InnoVen Capital",9000000.0,None,2021
15,Tessolve,1993,Bangalore,Electronics,Tessolve Semiconductor offers engineering in s...,"P Raja Manickam, Srinivas Chinamilli, Veerappan V",Novo Tellus Capital,40000000.0,None,2021
16,Smart Joules,2014,New Delhi,Renewable Energy,Smart Joules is an energy management company.,"Arjun P Gupta, Ujjal Majumdar, Sidhartha Gupta","Raintree Family Office, ADB arm",49000000.0,None,2021


Alright! These companies are listed on cruchbase, the leading provider of private-company prospecting and research solutions. Let's see if we can get reliable data to fill-in the nulls

In [37]:
data2021.iloc[1, 8] = 'Venture - Series Unknown'
data2021.iloc[5, 8] = 'Secondary Market'
data2021.iloc[6, 8] = 'Pre-Seed'
data2021.iloc[8, 8] = 'Seed'
data2021.iloc[9, 8] = 'Venture - Series Unknown'
data2021.iloc[11, 8] = 'Pre-Seed'
data2021.iloc[13, 8] = 'Series A'
data2021.iloc[14, 8] = 'Series D'
data2021.iloc[15, 8] = 'Series E'
data2021.iloc[16, 8] = 'Series A'
data2021.iloc[24, 8] = 'Debt Financing'
data2021.iloc[31, 8] = ' Series A'
data2021.iloc[34, 8] = 'Venture - Series Unknown'
data2021.iloc[35, 8] = 'Venture - Series Unknown'
data2021.iloc[36, 8] = 'Venture - Series Unknown'
data2021.iloc[37, 8] = 'Venture - Series Unknown'
data2021.iloc[40, 8] = 'Series A'
data2021.iloc[42, 8] = 'Seed'
data2021.iloc[46, 8] = 'Equity Crowdfunding'

Good! Let's have a look at what the unique values are again.

In [38]:
data2021['Stage'].unique()

array(['Pre-series A', 'Venture - Series Unknown', 'Series D', 'Series C',
       'Seed', 'Secondary Market', 'Pre-Seed', 'Series A', 'Series E',
       'Series B', 'Pre-seed', 'Debt Financing', ' Series A', None,
       'Equity Crowdfunding', 'Pre-series B', 'Debt', 'Bridge', 'Seed+',
       'Series F2', 'Series A+', 'Series G', 'Series F', 'Series H',
       'Series B3', 'PE', 'Series F1', 'Pre-series A1', 'Early seed',
       'Series D1', 'Seies A', 'Pre-series', 'Series A2', 'Series I'],
      dtype=object)

Ok, Let's put these values in a standard format.

In [39]:

def update_value(value):
    replacements = {
        r'Pre series|Early seed|Pre-series A|Pre-series A1': 'Pre-series',
        r'Seies A|Seed+|Pre-series B|Series A2': 'Series A',
        r'PE': 'Private Equity',
        r'Debt': 'Debt Financing',
        r'Seed1': 'Seed',
        r'None': 'Venture - Series Unknown',
        r'Series A+|Series B3|Series B+': 'Series B',
        r'Series F2|Series F1|Series D1|Series D|Series G|Series H|Series I|Series E|Series F': 'Series C',
    }
    
    for pattern, replacement in replacements.items():
        value = re.sub(pattern, replacement, str(value))
    
    return value

data2021 = data2021.applymap(update_value)


Great! Checking........

In [40]:
data2021['Stage'].unique()

array(['Pre-series', 'Venture - Series Unknown', 'Series C', 'Series B',
       'Secondary Market', 'Pre-Series B', 'Pre-seed',
       'Debt Financing Financing', ' Series B', 'Equity Crowdfunding',
       'Debt Financing', 'Bridge', 'Series B+', 'Private Equity',
       'Pre-series1'], dtype=object)

Ouch!  'Series B+',  'Pre-Series B' and 'Pre-series1' want to be treated diferently! I see. Let's implore another method!

In [41]:
data2021['Stage'].replace('Series B+', 'Series B', inplace=True)
data2021['Stage'].replace('Pre-series1', 'Pre-series', inplace=True)
data2021['Stage'].replace('Pre-Series B', 'Series A', inplace=True)


Nice! Let's take a look at the unique values for stage column again!

In [42]:
data2021['Stage'].unique()

array(['Pre-series', 'Venture - Series Unknown', 'Series C', 'Series B',
       'Secondary Market', 'Series A', 'Pre-seed',
       'Debt Financing Financing', ' Series B', 'Equity Crowdfunding',
       'Debt Financing', 'Bridge', 'Private Equity'], dtype=object)

In [43]:
data2021

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage,Year_Funded
0,Unbox Robotics,2019,Bangalore,AI startup,Unbox Robotics builds on-demand AI-driven ware...,"Pramod Ghadge, Shahid Memon","BEENEXT, Entrepreneur First",1200000.0,Pre-series,2021
1,upGrad,2015,Mumbai,EdTech,UpGrad is an online higher education platform.,"Mayank Kumar, Phalgun Kompalli, Ravijot Chugh,...","Unilazer Ventures, IIFL Asset Management",120000000.0,Venture - Series Unknown,2021
2,Lead School,2012,Mumbai,EdTech,LEAD School offers technology based school tra...,"Smita Deorah, Sumeet Mehta","GSV Ventures, Westbridge Capital",30000000.0,Series C,2021
3,Bizongo,2015,Mumbai,B2B E-commerce,Bizongo is a business-to-business online marke...,"Aniket Deb, Ankit Tomar, Sachin Agrawal","CDC Group, IDG Capital",51000000.0,Series C,2021
4,FypMoney,2021,Gurugram,FinTech,"FypMoney is Digital NEO Bank for Teenagers, em...",Kapil Banwari,"Liberatha Kallat, Mukesh Yadav, Dinesh Nagpal",2000000.0,Series B,2021
5,Urban Company,2014,New Delhi,Home services,Urban Company (Formerly UrbanClap) is a home a...,"Abhiraj Singh Bhal, Raghav Chandra, Varun Khaitan",Vy Capital,188000000.0,Secondary Market,2021
6,Comofi Medtech,2018,Bangalore,HealthTech,Comofi Medtech is a healthcare robotics startup.,Gururaj KB,"CIIE.CO, KIIT-TBI",200000.0,Series A,2021
7,Qube Health,2016,Mumbai,HealthTech,India's Most Respected Workplace Healthcare Ma...,Gagan Kapur,Inflection Point Ventures,3700000.0,Pre-series,2021
8,Vitra.ai,2020,Bangalore,Tech Startup,Vitra.ai is an AI-based video translation plat...,Akash Nidhi PS,Inflexor Ventures,3700000.0,Series B,2021
9,Taikee,2010,Mumbai,E-commerce,"Taikee is the ISO-certified, B2B e-commerce pl...","Nidhi Ramachandran, Sachin Chhabra",Venture - Series Unknown,1000000.0,Venture - Series Unknown,2021


['boAt', 'Beldara', 'Do Your Thng', 'Oye Rickshaw', 'Legitquest',
       'Swiggy', 'ShopSe', 'Fantasy Akhada', 'OfBusiness', 'ShareChat',
       'Safexpay', 'Speciale Invest', 'Meesho', 'CareerLabs', 'Chingari',
       'Onsurity', 'Karbon Card', 'Factors.AI', 'Elevar', 'Studio Sirah',
       'OTO Capital', '1Bridge', 'Skeps', 'Curefoods', 'TartanSense',
       'Bewakoof', 'Kirana247', 'NeoDocs', 'IsGoingOnline', 'Camp K12',
       'Elda Health', 'Sporjo', 'Advantage Club', 'SuperGaming', 'Ruptok',
       'O’ Be Cocktails', 'Imagimake', 'Hike', 'House of Kieraya',
       'goEgoNetwork', 'DrinkPrime', 'SATYA MicroCapital', 'Snack Amor',
       'Rapido', 'YPay', 'SleepyCat', 'Ultrahuman', 'UpScalio',
       'Yellow Class', 'Yojak', 'CreatorStack', 'Expertrons',
       'Rage Coffee', 'Klub', 'Navia Life Care', '1K Kirana Bazaar',
       'Zupee', 'Locale.ai', 'VerSe Innovation', 'MetroRide', 'Miko',
       'Whiz League', 'Hash', 'Unacademy', 'upGrad',
       'Stellaris Venture Partners', 'Celcius',
       'UrbanMatrix Technologies', 'StackBOX', 'BOX8', '8i Ventures',
       'Curefit', 'CHARGE+ZONE', 'Probus Insurance', 'PingoLearn',
       'Evenflow Brands', 'Practically', 'Atomberg', 'Ola', 'PropReturns',
       'Fitpage', 'ShopMyLooks', 'OfBusiness', 'Keka HR',
       'The Gourmet Jar', 'Flo Mobility', 'TheHouseMonk', 'Zepto',
       'Sirona Hygiene', 'Deciwood', 'Veefin', 'Karkinos Healthcare',
       'Zorgers', 'BangDB', 'Skippi Ice Pops', 'Mamaearth',
       'Vendor Infra', 'Hakuna Matata', 'OneCard', 'Hubhopper',
       'Comofi Medtech', 'Onelife', 'Marquee Equity', 'Avataar Ventures',
       'Codingal', 'Zingavita', 'TenderCuts', 'OwO',
       'Nazara Technologies', 'Scentials', 'GoTo', 'Remedico',
       'PrepBytes', 'SATYA MicroCapital', 'MPL', 'Taskmo',
       'MasterMentors', 'Ruptok', 'RevFin', 'Junio', 'Vista Rooms',
       'Makers Hive', 'MPL', 'Bombay Shaving Company', 'Furlenco',
       'Digit Insurance', 'MFine', 'Lohum', 'Darwinbox', 'True Elements',
       'SSA Finserv', 'Paperfly', 'Chalo', 'Udaan', 'Unacademy', 'Bolkar',
       'Knocksense', 'Pariksha', 'Zupee', 'Oneiric Gaming', 'Legalwiz.in',
       'Devic Earth', 'Pocket Aces', 'iMumz', 'BlackSoil', 'DcodeAI',
       'Biocon Biologics', 'Chai Waale', 'Biconomy', 'Slice',
       'JetSynthesys', 'MyGlamm', 'Gumlet', 'Bandhoo', 'ixigo', 'Droom',
       'Skymet', 'Mamaearth', 'Inspacco', 'GODI Energy', 'Lenskart',
       'Eka.care', 'Braingroom', 'Credenc', 'RiseBird', 'Vegrow',
       'Automovill', 'GalaxyCard', 'Bella Vita Organic', 'SmartCoin',
       'MYSUN', 'OneCode', 'Inshorts', 'Square Yards', 'OYO',
       'Slang Labs', 'Bikry app', 'Oliveboard', 'Kredent', 'SMOOR',
       'Pankhuri', 'UrbanKisaan', 'Vah Vah!', 'Sapio Analytics',
       'The Ayurveda Co', 'Pratilipi', 'Gourmet Garden',
       'Digit Insurance', 'Furlenco', 'Arcatron Mobility', 'TWID',
       'Pine Labs', 'Genworks Health', 'Pocketly', 'CoRover',
       'Rockclimber', 'SpEd@home', 'Green Soul', 'DealShare', 'Powerplay',
       'Accio Robotics', 'Good Health Clinic', 'KreditBee',
       'Onelife Nutriscience', 'Flatheads', 'Holisol', 'Now&Me',
       'Cora Health', 'Blaer Motors', 'Cashfree', 'Candes', 'MediBuddy',
       'Capital Float', 'Shyplite', 'CustomerGlu', 'Cell Propulsion',
       'Wellbeing Nutrition', 'Chqbook', "BYJU'S", 'Power Gummies',
       'WaterScience', 'Juicy chemistry', 'Samaaro', 'PazCare',
       'Answer Genomics', 'BYJU’S', 'India Quotient', 'BigLeap',
       'MicroDegree', 'Plutomen', 'Grinntech', 'Nobel Hygiene',
       'LegalPay', 'Lavado', 'Navars', 'Edmingle', 'Slice', 'NIRAMAI',
       'Detect Technologies', 'CredR', 'Fourth Partner Energy',
       'Nalanda Learning', 'Knackit', 'Raptee Energy', 'Meddo',
       'Inshorts', 'Dream Sports', 'Safex Chemicals', 'Instoried',
       'Bombay Shaving Company', 'Homingos', 'Anar Business Community',
       'Five Star Finance', 'Annapurna Finance', 'Bewakoof', 'IndiaLends',
       'Tilt', 'Zaara Biotech', 'Purplle', 'NewLink Group',
       'Nazara Technologies', 'Policybazaar', 'Livve Homes',
       'Svasti Microfinance', 'Nexprt', 'Wysa', 'Ankur capital',
       'Indic Inspirations', 'BlackSoil NBFC', 'OYO', 'Max Healthcare',
       'Kinara Capital', 'NODWIN', 'Ecom Express', 'IGL', 'AMPM',
       'Cashify', 'Bijnis', 'Blume Ventures', 'ImaginXP', 'Virohan',
       'Apna.co', 'ideaForge', 'Clairco', 'Lenskart',
       'Asirvad Microfinance', 'Clensta', 'Polygon', 'Get My Parking',
       'FanCode', 'ThatMate', 'Disruptium', 'Pickright Technologies',
       'Enthu.ai', '6Degree', "BYJU'S", 'Thingsup', 'Zepto', 'TurboHire',
       'SatSure', 'VilCart', 'TRDR', 'Zoomcar', 'Leap India',
       'SuperBottoms', 'The Viral Fever', 'Knocksense', 'Petpooja',
       'Wingreens Farms', 'Doola', 'Bombay Hemp Company', 'Toplyne',
       'Wonderchef', 'Better Capital', 'Tickertape', 'Zenpay Solutions',
       'Arbo Works', 'Defy', 'Homversity', 'True Balance', 'Recordent',
       'Indifi', 'Dream Sports', 'Rentomojo', 'Loop Health', 'Smartstaff',
       'Mobileware Technologies', "BYJU'S", 'Hyperface',
       'Kaar Technologies', 'Kissan Pro', 'LeadSquared', 'Phool.co',
       'Log 9 Materials', 'Chingari', 'Ola Electric', 'Visit Health',
       'Zetwerk', 'Wiingy', 'Melorra', 'Arcana', 'VLCC Health Care',
       'Duroflex', 'Onato', 'Mestastop Solutions', 'Totality',
       'SUN Mobility', 'MergerDomo', 'Tvasta', 'Trell',
       'The Indus Valley', 'BharatPe', 'BankSathi', 'Gramophone',
       'Shumee', 'Instoried', 'R for Rabbit', 'Vakilsearch', 'Purplle',
       'Wallmantra', 'Meesho', 'PumPumPum', 'Homeville', 'Ola Electric',
       'EV Plugs', 'CredRight', 'Auntie Fung', 'Delhivery',
       'Sanctum Wealth', 'Upgame', 'Sterling Accuris Wellness',
       'Capital Float', 'Alpha Coach', 'byteXL', 'Design Cafe',
       'Expertrons', 'Supertails', 'LifeCell', 'Leverage Edu', 'Enercomp',
       'Medpho', 'LivQuik Technology', 'EventBeep', 'Easiloan',
       'Tinkerly', 'Boutique Spirit Brands', 'eShipz', 'Pine Labs',
       'Atomberg Technologies', 'Genext Students', 'GameEon Studios',
       'Lido Learning', 'Peppermint', 'Sugar.fit', 'Geniemode',
       'Sapio Analytics', 'Voxelgrids']